**Importar Librerías**

In [4]:
import pandas as pd
from pandas import datetime
from matplotlib import pyplot
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential    
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from keras.models import Model
from keras.layers import Input
from keras.layers.merge import concatenate

from sklearn.metrics import mean_squared_error   # para calcular el error cuadratico medio
from math import sqrt

from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

**Preprocesing Data - Functions:**
1. Drop outliers.
2. Standardizing values.
3. Dealing with missing values. 
4. Feature Engineering - Differencing.
5. Feature Engineering - Computing Time Features.

In [5]:
#Remove Outliers:
def remove_outliers(dataframe):
  data = dataframe.copy()
  data.drop(columns=['Latitud', 'Longitud', 'Fecha', 'Ruido (dB)', 'UV', 'Presion (hPa)'], inplace=True)
  for (columnName, columnData) in data.iteritems():
      feature = data[columnName]
      #Search for outliers:
      outliers = feature.between(feature.quantile(.1), feature.quantile(.9))
      index_names = data[~outliers].index
      #Replace outliers with word "NaN":
      data[columnName].loc[index_names] = np.nan
      #print(outliers.value_counts())
      #print(index_names)
  return data

In [6]:
#Computing Standard Vector.
from sklearn.preprocessing import StandardScaler
def standarized_vector(dataframe):
  data = dataframe.copy()
  data.drop(columns=['Latitud', 'Longitud', 'Fecha'], inplace=True)
  vector_scaler = StandardScaler()
  vector_scaler.fit_transform(data)
  return vector_scaler

In [7]:
#Standarizing Values.
def standarized_values(dataframe, vector_scaler):
  data = dataframe.copy()
  data.drop(columns=['Latitud', 'Longitud', 'Fecha'], inplace=True)
  df_standard_scale = pd.DataFrame(vector_scaler.transform(data))
  df_standard_scale = df_standard_scale.rename(columns={0: "CO (ug/m3)", 1: "H2S (ug/m3)", 2: "NO2 (ug/m3)", 3: "O3 (ug/m3)", 4: "PM10 (ug/m3)", 5: "PM2.5 (ug/m3)", 6: "SO2 (ug/m3)", 7: "Ruido (dB)", 8: "UV", 9: "Humedad (%)", 10: "Presion (hPa)", 11: "Temperatura (C)"})
  return df_standard_scale

In [8]:
#Interpolation of missing values:
def deal_missing_values(dataframe):
  data = dataframe.copy()
  for (columnName, columnData) in data.iteritems():
    feature = data[columnName]
    data[columnName] = feature.interpolate()
  return data

In [9]:
#Time differencing (t-1):
def differencing_transform(dataframe):
  data = dataframe.copy()
  for (columnName, columnData) in data.iteritems():
    data_dif_time = data[columnName]
    data[columnName] = data_dif_time - data_dif_time.shift(1)
  return data

In [10]:
# Generate multivariate features from n_steps of a group of parallel secuences.
def generate_time_features(dataframe, n_steps_in, n_steps_out):
  x, y = [], []
  for i in range(len(dataframe.index)):
      end_step = i + n_steps_in
      out_end_ix = end_step + n_steps_out
      if out_end_ix > len(dataframe.index)-1:
        break
      # Separate time feature values from time target values.
      x.append(dataframe.iloc[i:end_step, 0:6].values)
      # Get only pollutants for target.
      y.append(dataframe.iloc[end_step:out_end_ix, 0:6].values)
  return np.array(x), np.array(y)

In [11]:
# Split Train/Test dataset:
def split_train_test(x_time_features, y_time_targets):
  X_train, X_test, y_train, y_test = train_test_split(x_time_features, y_time_targets, test_size=0.15, random_state=42, shuffle=True)
  return X_train, X_test, y_train, y_test

In [12]:
# Plot variables data.
def plot_features(dataset):
  for (columnName, columnData) in dataset.iteritems():
    plt.figure()
    print(columnName)
    dataset[columnName].plot(figsize=(10,10),grid =True)
    plt.show()

In [13]:
# Search the interval of time that has constant repetead values.
def find_constantvalue_intervals(dataset, n_constant_max, interlap_max):
  check_points = []
  point_break = []
  # Search for points of time (intervals) that have the same value over "n_constant_max" times.
  for (columnName, columnData) in dataset.iteritems():
    i = 0
    while i <= (len(columnData.index)):
      constant_count = 0
      for j in range(1,len(columnData.index)):
        if i+j > len(columnData.index)-1:
          break
        # If the next value is the same.
        if columnData[i] == columnData[i+j]:
          constant_count = constant_count + 1
        # If the next value is different.
        else:
          break
      # If number of times value was repeated was more than "n_constant_max" times (24 points / 1 day).
      if constant_count >= n_constant_max:
        point_break.append((i,i+constant_count))
        i = i + constant_count
      else:
        i = i + 1
  # Eliminate duplicate intervals between variables.
  newlist = []
  for i in point_break:
      if i not in newlist:
        newlist.append(i)
        newlist = sorted(newlist)
  # Eliminate intervals that are included in others.
  limits = newlist.copy()
  for intervals1 in limits:
    for intervals2 in limits:
      if intervals2 != intervals1:
        if ((intervals1[0] <= intervals2[0]) and (intervals1[1] >= intervals2[1])):
          limits.remove(intervals2)
      limits = sorted(limits)
  # Generate union of intervals if there is an intersection between them.
  constructs = []
  checkpoints = limits.copy()
  #Iterate over intersects of intervals.
  for bis in range(int(len(limits))):
    for intervals1 in checkpoints:
      for intervals2 in checkpoints:
        if intervals2 != intervals1:
          if (intervals1[1] >= intervals2[0] and intervals1[1] <= intervals2[1]):
            construct = (intervals1[0],intervals2[1])
            constructs.append(construct)
            if intervals1 in checkpoints:
              checkpoints.remove(intervals1)
            if intervals2 in checkpoints:
              checkpoints.remove(intervals2)
            checkpoints.extend(constructs)
            constructs = []
            checkpoints = sorted(checkpoints)
  # Join intervals with are separeted less than "interlap_max" times (n_steps_input + n_steps_output)
  backup = checkpoints.copy()
  for bis in range(int(len(backup)/2)):
    i = 0
    clean_checkpoints = []
    while ((i+1) <= len(checkpoints)-1):
      if ((checkpoints[i+1][0] - checkpoints[i][1]) <= interlap_max):
        clean_checkpoints.append((checkpoints[i][0],checkpoints[i+1][1]))
        i = i + 2
        flag = 1
      else:
        clean_checkpoints.append(checkpoints[i])
        i = i + 1
        flag = 0
    if flag == 0:
      clean_checkpoints.append(checkpoints[len(checkpoints)-1])
    checkpoints = clean_checkpoints

  print("Cantidad de Intervalos:", len(checkpoints))
  print("Los intervalos de información constante son:", checkpoints)
  return checkpoints

In [47]:
# Time difference for time features vectors:
def time_difference_features(X_features, y_target):
  Y_sub, X_sub = [], []
  Y_target_differences, X_features_differences = [], []
  # Iterate between all the time features.
  for i in range(len(X_features)):
    j=0
    X_sub, Y_sub =  [], []
    # Calculate first time difference for the target vector (First Value from Target Vector - Last Value from Feature Vector)
    difference = y_target[i][0] - X_features[i][len(X_features[0][0])-1]
    # Store difference result in array of arrays.
    Y_sub.append(difference)
    while ((j+1) <= (len(X_features[0])-1)):
      # Calculate time difference from the feature vector (Value from Target Vector(t) - Value from Target Vector(t-1))
      difference = X_features[i][j+1] - X_features[i][j]
      # Store difference result in array of arrays.
      X_sub.append(difference)
      # Take into account that target vector has less elements than features vector.
      if ((j+1) <= (len(y_target[0])-1)):
        # Calculate time difference from the target vector (Value from Feature Vector(t) - Value from Target Vector(t-1))
        difference = y_target[i][j+1] - y_target[i][j]
        # Store difference result in array of arrays.
        Y_sub.append(difference)
      j=j+1
    # Store result in array of arrays.
    X_features_differences.append(X_sub)
    # Store result in array of arrays.
    Y_target_differences.append(Y_sub)

  return np.array(X_features_differences), np.array(Y_target_differences)

In [14]:
# Generate multivariate features from n_steps with the first next hour for persistent model from y_test.
def generate_persistent_features_test(y_test, n_steps_out):
  y_persistent_test = []
  for i in range(y_test.shape[0]):
    next_hour = np.zeros((n_steps_out, 6))
    next_hour[:,:] = y_test[i][0]
    y_persistent_test.append(next_hour)

  return np.array(y_persistent_test)

In [16]:
# Generate multivariate features from n_steps with the first next hour for persistent model.
def generate_persistent_features(dataframe, n_steps_in, n_steps_out):
  x, y = [], []
  for i in range(len(dataframe.index)):
      end_step = i + n_steps_in
      out_end_ix = end_step + n_steps_out
      if out_end_ix > len(dataframe.index)-1:
        break
      # Separate time feature values from time target values.
      x.append(dataframe.iloc[i:end_step, :].values)
      # Get only the next hour pollutants for target.
      next_hour = np.zeros((n_steps_out, 6))
      next_hour[:,:] = dataframe.iloc[end_step, 0:6].values
      y.append(next_hour)
      
  return np.array(x), np.array(y)

In [48]:
# Generate train time features dataset from all qHAWAX stations.
n_steps_in = 24
n_steps_out = 12
total_time_features = 0
total_time_targets = 0
n_stations = 10
month_from_test = 10
n_constant_max = 24
interlap_max = n_steps_out + n_steps_in

for i in range(n_stations):
  # Change name of file to read.
  station_file_number = str(i+1)
  station_file_name = "qHAWAX-VariablesContaminacion_1 (" + station_file_number + ").csv"
  # Read CSV file.
  dataset = pd.read_csv(station_file_name)
  # Remove outliers.
  data_outliers = remove_outliers(dataset)
  # Generate Train time features from time windowing process.
  iter, flag = 0, 0
  # Search for the intervals in which the data remains constant (same point of data is repeated for more than "n_constant_max" times);
  # Intervals which have less than "interlap_max" values between each of them.
  checkpoints = find_constantvalue_intervals(data_outliers, n_constant_max, interlap_max)
  data_interval, total_x_station, total_y_station = [], [], []
  # If there are intervals of time where the data is constant (no accurate):
  if len(checkpoints) > 0:
    # First interval of data (from 0 to first checkpoint (A,B) --> Interval = (0,A)).
    interval = (0,checkpoints[0][0])
    # Do not generate time features if there are not enough registers in the interval.
    if (interval[1] - interval[0]) >= interlap_max:
      # Build interval of useful data.
      data_interval = data_outliers[interval[0]:interval[1]].copy()
      # Deal with missing values with interpolation.
      data_miss = deal_missing_values(data_interval)
      # Generate time features from slice of data.
      x_time_features_station, y_time_targets_station = generate_time_features(data_miss, n_steps_in, n_steps_out)
      # If there are time features in the first interval.
      if (len(x_time_features_station) > 0):
        total_x_station = x_time_features_station
        total_y_station = y_time_targets_station
        flag = 1

    while ((iter+1) <= len(checkpoints)-1):
      # Interval of data between checkpoints: (A,B);(C,D) --> Interval = (B,C).
      interval = (checkpoints[iter][1], checkpoints[iter+1][0])
      # Do not generate time features if there are not enough registers in the interval.
      if (interval[1] - interval[0]) >= interlap_max:
        # Build interval of useful data.
        data_interval = data_outliers[interval[0]:interval[1]].copy()
        # Deal with missing values with interpolation.
        data_miss = deal_missing_values(data_interval)
        # Generate time features from slice of data.
        x_time_features_station, y_time_targets_station = generate_time_features(data_miss, n_steps_in, n_steps_out)
        # If there were NO time features in the first interval but there are in the subsequent intervals.
        if ((len(x_time_features_station) > 0) and (flag != 1)):
          total_x_station = x_time_features_station
          total_y_station = y_time_targets_station 
          flag = 1
        # If there were time features in the first interval and there are in the subsequent intervals.
        elif ((len(x_time_features_station) > 0) and (flag == 1)):
          total_x_station = np.concatenate((total_x_station, x_time_features_station), axis=0)
          total_y_station = np.concatenate((total_y_station, y_time_targets_station), axis=0)
      # Iterate through all the time periods with useful data.        
      iter = iter + 1

    # Last interval of data (from last checkpoint to last dataset register: (C,D) --> Interval = (D,#registros)).  
    interval = (checkpoints[len(checkpoints)-1][1], len(data_outliers)-1)
    # Do not generate time features if there are not enough registers in the interval.
    if (interval[1] - interval[0]) >= interlap_max:
      # Build interval of useful data.
      data_interval = data_outliers[interval[0]:interval[1]].copy()
      # Deal with missing values with interpolation.
      data_miss = deal_missing_values(data_interval)
      # Generate time features from slice of data.
      x_time_features_station, y_time_targets_station = generate_time_features(data_miss, n_steps_in, n_steps_out)
      # If there were NO time features in the first interval but there are in the subsequent intervals.
      if ((len(x_time_features_station) > 0) and (flag != 1)):
        total_x_station = x_time_features_station
        total_y_station = y_time_targets_station
        flag = 1
      # If there were time features in the first interval and there are in the subsequent intervals.
      elif ((len(x_time_features_station) > 0) and (flag == 1)):
        total_x_station = np.concatenate((total_x_station, x_time_features_station), axis=0)
        total_y_station = np.concatenate((total_y_station, y_time_targets_station), axis=0) 
  # If there are NO intervals of time where the data is constant (no accurate):
  else:
    total_x_station, total_y_station = generate_time_features(data_outliers, n_steps_in, n_steps_out)
  # If it is the first generation of time features.
  if (i == 0):
    total_time_features = total_x_station
    total_time_targets = total_y_station
  # If it is not the first generation of time features.
  else:
    total_time_features = np.concatenate((total_time_features, total_x_station), axis=0)
    total_time_targets = np.concatenate((total_time_targets, total_y_station), axis=0)

# Print results for Features & Target.
print("-------------------- Features & Target Dimensions --------------------")
print("Size of time features data array:", total_time_features.shape)
print("Size of time target data array:", total_time_targets.shape)

# Split Train/Test.
X_train, X_test, y_train, y_test = split_train_test(total_time_features, total_time_targets)

# Print results after Train/Test.
print("-------------------- Results after Train/Test Split --------------------")
print("Size of time features TRAIN data array:", X_train.shape)
print("Size of time features TEST data array:", X_test.shape)
print("Size of time target TRAIN data array:", y_train.shape)
print("Size of time target TEST data array:", y_test.shape)

# Time Differences.
X_train_difference, y_train_difference = time_difference_features(X_train, y_train)
X_test_difference, y_test_difference = time_difference_features(X_test, y_test)

# Print results after Time Difference Train/Test.
print("-------------------- Results after Time Difference Train/Test Split --------------------")
print("Size of time features TRAIN data array:", X_train_difference.shape)
print("Size of time features TEST data array:", X_test_difference.shape)
print("Size of time target TRAIN data array:", y_train_difference.shape)
print("Size of time target TEST data array:", y_test_difference.shape)

# Stadarizing values.
# data_train_scaled, robust_scaled_vector = robust_scaler(X_train, y_train)

#plot_features(data_outliers)

Cantidad de Intervalos: 4
Los intervalos de información constante son: [(1403, 1447), (1573, 1613), (1747, 1780), (3102, 3129)]
Cantidad de Intervalos: 1
Los intervalos de información constante son: [(80, 104)]
Cantidad de Intervalos: 4
Los intervalos de información constante son: [(92, 157), (332, 399), (545, 587), (735, 2728)]
Cantidad de Intervalos: 10
Los intervalos de información constante son: [(10, 1578), (1793, 1904), (2019, 2255), (2305, 2614), (2659, 2902), (3005, 3041), (3212, 4076), (4212, 4497), (4601, 4689), (4750, 5173)]
Cantidad de Intervalos: 17
Los intervalos de información constante son: [(1071, 1112), (1382, 1447), (1529, 1606), (1743, 1782), (1914, 1950), (2075, 2116), (2247, 2283), (2414, 2450), (3097, 3152), (3585, 3617), (3753, 3786), (3919, 3954), (4082, 4116), (4583, 4626), (4743, 4779), (4911, 4947), (5076, 5114)]
Cantidad de Intervalos: 0
Los intervalos de información constante son: []
Cantidad de Intervalos: 5
Los intervalos de información constante son: [(

**Modelo de Persistencia y Cálculo de RMSE**

In [ ]:
y_persistent_test = generate_persistent_features_test(y_test, n_steps_out)
y_persistent_test.shape

sum_rmse = 0
rmse_per_variable = []
for row in range(y_test.shape[0]):  # por cada semana predicha
  for col in range(y_test.shape[1]): # por cada dia
    sum_rmse += (y_test[row][col] - y_persistent_test[row][col])**2

for i in range(len(sum_rmse)):
  rmse_per_variable.append(sqrt(sum_rmse[i] / (y_test.shape[0] * y_test.shape[1])))

rmse_per_variable

[nan, nan, nan, nan, nan, nan]

**NN Architecture:**
- Multi-variate.
- Multiple steps.

In [50]:
x_train = X_train_difference
# Aplana las secuencias de salida
n_output = y_train_difference.shape[1] * y_train_difference.shape[2] # numero de salidas
y_train = y_train_difference.reshape((y_train_difference.shape[0], n_output))

In [52]:
#Topología de CNN
n_kernel = 3
n_filters = 100
# obtiene el numero de series (tercera adimension de X)
n_features = x_train.shape[2]
n_steps_in_difference = n_steps_in - 1
# define arquitectura de modelo MLP
modelo = Sequential()
modelo.add(Conv1D(filters=n_filters, kernel_size=n_kernel, activation='relu', input_shape=(n_steps_in_difference, n_features)))
modelo.add(Conv1D(filters=n_filters, kernel_size=n_kernel, activation='relu'))
modelo.add(MaxPooling1D(pool_size=2))
modelo.add(Conv1D(filters=n_filters, kernel_size=n_kernel, activation='relu'))
modelo.add(MaxPooling1D(pool_size=2))
modelo.add(Flatten())
modelo.add(Dense(100, activation='relu'))
modelo.add(Dense(n_output))
modelo.compile(loss='mse', optimizer='adam')

# entrena el modelo MLP con la la data de entrenamiento generada
modelo.fit(x_train, y_train, epochs=1000, verbose=0)

# muestra un resumen de la topologia del modelo
modelo.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 21, 100)           1900      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 19, 100)           30100     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 9, 100)            0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 7, 100)            30100     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 3, 100)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)              

In [73]:
n_features = X_test_difference.shape[2]
n_steps_in_difference = X_test_difference.shape[1]

X_input = X_test_difference[0]

X_input = X_input.reshape(1, n_steps_in_difference, n_features)

yhat = modelo.predict(X_input, verbose=0)
yhat

array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan]], dtype=float32)

In [ ]:
#Pendientes
- Modelo Base de Persistencia. - (OK)
- Función para evaluar desempeño de NN (RMSE) - (OK)
- Normalización Robusta de Parámetros
- ¿Qué hacer con los valores NaN?
- CNN --> Conv2D?

- Eliminar en base a outliers de boxplot (ecuación de cuartiles)
- Imputar los valores <6 con interpolación lineal
- Valores nulos > 20 no considerarlos (calcular time features por bloque)
- Time differences en persisntencia debe ser 0
- trabajar todo en original y el paso de diferenciación en data 

SyntaxError: ignored